In [1]:
import glob
import codecs
import pandas as pd

In [2]:
import bz2
import cPickle as pickle
import time

In [3]:
from preprocessing import standoff_to_entitydf, entitydf_to_meshdict, entitydf_to_tagdict

# Standoff format to entity DF

In [4]:
from preprocessing import standoff_to_entitydf

In [5]:
ann_files = []
ann_files.extend(glob.glob("/home/antonis/data/biocreative6/goldset/*/*.ann"))
ann_files.extend(glob.glob("/home/antonis/data/biocreative6/NCBI_parsed/outgoing_citations/*.ann"))

In [6]:
ann_files[0:3]+ann_files[-5:]

['/home/antonis/data/biocreative6/goldset/development/18480678.ann',
 '/home/antonis/data/biocreative6/goldset/development/7703721.ann',
 '/home/antonis/data/biocreative6/goldset/development/23066089.ann',
 '/home/antonis/data/biocreative6/NCBI_parsed/outgoing_citations/18155073.ann',
 '/home/antonis/data/biocreative6/NCBI_parsed/outgoing_citations/4087830.ann',
 '/home/antonis/data/biocreative6/NCBI_parsed/outgoing_citations/19246362.ann',
 '/home/antonis/data/biocreative6/NCBI_parsed/outgoing_citations/14561769.ann',
 '/home/antonis/data/biocreative6/NCBI_parsed/outgoing_citations/9895047.ann']

In [8]:
start_time = time.time()

# parse standoff annotations again !! Time consuming 
entities = standoff_to_entitydf(ann_files, )
# Write to bz2 pickle 
with bz2.BZ2File('/home/antonis/data/biocreative6/entities/entitiesdf_GS+NCBI_outg_citations.pkl.bz2', 'wb') as f:
    pickle.dump(entities, f)

print time.time()-start_time

Found 22693 standoff files
Processing file 0 of 22693 (0.0 min)
Processing file 500 of 22693 (0.0 min)
Processing file 1000 of 22693 (0.1 min)
Processing file 1500 of 22693 (0.2 min)
Processing file 2000 of 22693 (0.3 min)
Processing file 2500 of 22693 (0.4 min)
Processing file 3000 of 22693 (0.6 min)
Processing file 3500 of 22693 (0.9 min)
Processing file 4000 of 22693 (1.2 min)
Processing file 4500 of 22693 (1.5 min)
Processing file 5000 of 22693 (1.9 min)
Processing file 5500 of 22693 (2.4 min)
Processing file 6000 of 22693 (2.9 min)
Processing file 6500 of 22693 (3.4 min)
Processing file 7000 of 22693 (4.0 min)
Processing file 7500 of 22693 (4.7 min)
Processing file 8000 of 22693 (5.4 min)
Processing file 8500 of 22693 (6.1 min)
Processing file 9000 of 22693 (6.9 min)
Processing file 9500 of 22693 (7.8 min)
Processing file 10000 of 22693 (8.7 min)
Processing file 10500 of 22693 (9.6 min)
Processing file 11000 of 22693 (10.6 min)
Processing file 11500 of 22693 (11.7 min)
Processing 

# ~~~~

# Load entities df

In [9]:
with bz2.BZ2File('/home/antonis/data/biocreative6/entities/entitiesdf_GS+NCBI_outg_citations.pkl.bz2', 'rb') as f:
    entities = pickle.load(f)

In [10]:
entities.head()

,T,EntityType,start,stop,EntityText,_,identifier,doc_id
0,T10,Chemical,1629,1632,MPH,,MESH:C041626,18480678
1,T11,Chemical,1707,1712,d-MPH,,Chemical:1,18480678
2,T12,Chemical,1764,1769,l-MPH,,Chemical:2,18480678
3,T13,Chemical,1885,1904,d-enantiomer of MPH,,Chemical:3,18480678
4,T14,Chemical,285,293,dopamine,,MESH:D004298,18480678


### Check mapped vs un-mapped entities in our dataset

In [14]:
154228+ 26964+136649+13478-41200


290119

In [11]:
# un-mapped Chemicals (excluding tmTool unknown: identifier)
def check_unmapped(ent_type, entities):
    ser = pd.Series(map(lambda x: x.startswith(ent_type+":"),entities[entities.EntityType==ent_type]['identifier'] )).value_counts().rename({False: "Mapped", True: "Un-mapped"},)
    return ser

for ent_type in entities.EntityType.unique():
    print ent_type
    print check_unmapped(ent_type, entities),'\n'
    
# Previous results (GS + chemdner_silver)
# Chemical
# Mapped       67113
# Un-mapped    30656
# dtype: int64 

# Gene
# Mapped       51594
# Un-mapped    17974
# dtype: int64 

# Disease
# Mapped       83511
# Un-mapped        3
# dtype: int64 

# Species
# Mapped    55273
# dtype: int64 

# Mutation
# Mapped    1690
# dtype: int64 

Chemical
Mapped       154228
Un-mapped     26964
dtype: int64 

Gene
Mapped       136649
Un-mapped     13478
dtype: int64 

Species
Mapped    50508
dtype: int64 

Disease
Mapped       69080
Un-mapped        5
dtype: int64 

Mutation
Mapped    3642
dtype: int64 



# Create entities import files for snorkel (candidate extraction)

### Create tag dict

In [15]:
unary_tags = entitydf_to_tagdict(entities)
# unary_tags

In [16]:
len(unary_tags)

22040

In [17]:
unary_tags[unary_tags.keys()[1]]

{(u'Chemical|MESH:D010755', 299, 315),
 (u'Chemical|MESH:D012524', 144, 149),
 (u'Chemical|MESH:D012524', 449, 454),
 (u'Disease|unknown:D006331', 67, 88),
 (u'Disease|unknown:D006331', 1030, 1051),
 (u'Disease|unknown:D006331', 1767, 1788),
 (u'Disease|unknown:D012131', 1103, 1122),
 (u'Disease|unknown:D012640', 90, 98),
 (u'Disease|unknown:D012640', 1068, 1076),
 (u'Disease|unknown:D064420', 1372, 1377),
 (u'Disease|unknown:D064420', 1793, 1808),
 (u'Disease|unknown:D066126', 1232, 1256),
 (u'Gene|NCBIGENE:43', 914, 934),
 (u'Gene|NCBIGENE:590', 30, 51),
 (u'Gene|NCBIGENE:590', 169, 190),
 (u'Gene|NCBIGENE:590', 195, 199),
 (u'Gene|NCBIGENE:590', 385, 389),
 (u'Gene|NCBIGENE:590', 568, 572),
 (u'Gene|NCBIGENE:590', 607, 611),
 (u'Gene|NCBIGENE:590', 939, 943),
 (u'Gene|NCBIGENE:590', 1165, 1169),
 (u'Gene|NCBIGENE:590', 1336, 1340),
 (u'Gene|NCBIGENE:590', 1579, 1583),
 (u'Gene|NCBIGENE:590', 1604, 1608),
 (u'Gene|NCBIGENE:590', 1703, 1707),
 (u'Species|NCBITaxon:9823', 1550, 1554)}

#### Write to bz2 pickle

In [18]:
# Write to bz2 pickle 
with bz2.BZ2File('/home/antonis/data/biocreative6/entities/unary_tags.pkl.bz2', 'w') as f:
    pickle.dump(unary_tags, f)

### Create mesh dict

In [19]:
mesh_dict = entitydf_to_meshdict(entities)

In [20]:
# Write to bz2 pickle 
with bz2.BZ2File('/home/antonis/data/biocreative6/entities/mesh_dict.pkl.bz2', 'w') as f:
    pickle.dump(tuple([mesh_dict['Chemical'], mesh_dict['Gene']]), f)

# Create relationship import files for snorkel (candidate extraction)

### Create goldset relationships dict
** Skipped - assumes document-level annotations **

In [21]:
# from preprocessing import gold_relations_to_dict
T_mapping = entities.groupby('doc_id')[['T','identifier']].apply(lambda g: dict(g.values.tolist())).to_dict()

** To tsv ** 

In [22]:
# from preprocessing import gold_relations_to_tsv

In [23]:
entities.head()

,T,EntityType,start,stop,EntityText,_,identifier,doc_id,tup
0,T10,Chemical,1629,1632,MPH,,MESH:C041626,18480678,"(Chemical|MESH:C041626, 1629, 1632)"
1,T11,Chemical,1707,1712,d-MPH,,Chemical:1,18480678,"(Chemical|Chemical:1, 1707, 1712)"
2,T12,Chemical,1764,1769,l-MPH,,Chemical:2,18480678,"(Chemical|Chemical:2, 1764, 1769)"
3,T13,Chemical,1885,1904,d-enantiomer of MPH,,Chemical:3,18480678,"(Chemical|Chemical:3, 1885, 1904)"
4,T14,Chemical,285,293,dopamine,,MESH:D004298,18480678,"(Chemical|MESH:D004298, 285, 293)"


In [24]:
entities.tail()

,T,EntityType,start,stop,EntityText,_,identifier,doc_id,tup
454549,T42,Chemical,1823,1827,PGE2,N40\tReference T42 MESH:D015232\tPGE2,MESH:D015232,9895047,"(Chemical|MESH:D015232, 1823, 1827)"
454550,T43,Chemical,1906,1914,Nicotine,N41\tReference T43 MESH:D009538\tNicotine,MESH:D009538,9895047,"(Chemical|MESH:D009538, 1906, 1914)"
454551,T44,Disease,1955,1959,TNBS,N42\tReference T44 unknown:D000592\tTNBS,CHEBI:53063,9895047,"(Disease|CHEBI:53063, 1955, 1959)"
454552,T45,Disease,1968,1975,colitis,N43\tReference T45 unknown:D003092\tcolitis,unknown:D003092,9895047,"(Disease|unknown:D003092, 1968, 1975)"
454553,T46,Chemical,2030,2034,PGE2,N44\tReference T46 MESH:D015232\tPGE2,MESH:D015232,9895047,"(Chemical|MESH:D015232, 2030, 2034)"


In [25]:
from collections import defaultdict

In [26]:
# prepare for extracting to tsv

entities['startstop'] = map(lambda start,stop: [int(start), int(stop)], entities.start,entities.stop)
entities = entities[map(lambda x: x in ['Chemical', 'Gene'],entities.EntityType)] #keep only Chemicals & Genes (combinations get too big)
# dict of offsets per T
T_offsets = entities.groupby('doc_id')[['T','startstop']].apply(lambda g: dict(g.values.tolist())).to_dict()
# dict of T types (to avoid pointless combinations)
T_types = defaultdict(lambda: defaultdict(list))
for row in entities.iterrows():
    T_types[row[1].doc_id][row[1]['EntityType']].append(row[1]['T'])

In [7]:
from preprocessing import gold_relations_to_tsv

In [28]:
# Analytics relationships 

input_path = glob.glob("/home/antonis/data/biocreative6/tsv/*/*gold_standard.tsv")
#, output_path, T_offsets, T_types, 
true_rels=['CPR:3', 'CPR:4']
# correct_last_offset = True

relationships = pd.DataFrame()
if isinstance(input_path,str):
    input_path = [input_path]
for path in input_path:
    relationships = relationships.append(pd.read_table(path, names = ['doc_id', 'CPR', 'arg1', 'arg2']) , ignore_index=True)

# convert doc_id to string
relationships['doc_id'] = map(lambda x: str(x) , relationships['doc_id'])
relationships['label'] = map(lambda x: 1 if x in true_rels else -1, relationships.CPR)
print relationships.label.value_counts()

# Total (+) relationships in gold set:
# Train: 3022
# Dev: 1644
# Test 2326
# Total: 6992


 1    6992
-1    3039
Name: label, dtype: int64


In [29]:
# neg_rels1 = pd.DataFrame(map(lambda (T1,T2): (doc_id,-1,T1,T2) , product(T_types[doc_id]['Chemical'],T_types[doc_id]['Gene'])) , columns = [u'doc_id', u'CPR', u'arg1', u'arg2'])

# neg_rels1['arg1'] = map( lambda arg: "Arg1:"+arg , neg_rels1['arg1'])
# neg_rels1['arg2'] = map( lambda arg: "Arg2:"+arg , neg_rels1['arg2'])

# neg_rels

In [30]:
# Extract rels and save to disk

In [31]:
path = glob.glob("/home/antonis/data/biocreative6/tsv/*/*gold_standard.tsv")
print 'Converting labels from:\n'+ ",\n".join(path)

print 'Labels statistics:'
gold_relations_to_tsv(path,'/home/antonis/data/biocreative6/entities/gold_rels_complete.tsv',T_offsets, T_types)

Converting labels from:
/home/antonis/data/biocreative6/tsv/development/chemprot_development_gold_standard.tsv,
/home/antonis/data/biocreative6/tsv/test_gs/chemprot_test_gold_standard.tsv,
/home/antonis/data/biocreative6/tsv/training/chemprot_training_gold_standard.tsv
Labels statistics:
-1    314556
 1      6987
Name: label, dtype: int64
